## Представление времени

1. Строка: [ISO 8601](https://en.wikipedia.org/wiki/ISO_8601)
    * 2021-10-01 18:00:01
    * 2021-10-01T18:00:01
    * 2021-10-01T18:00:01+05:00
2. Таймстемп: [unix time](https://en.wikipedia.org/wiki/Unix_time), [converter](https://www.epochconverter.com/)
    * 1728391229
    * Как иметь больше чем секундную точность?
 
**Unix-время** - количество секунд, прошедших с полуночи (00:00:00 UTC) 1 января 1970 года; этот момент называют Unix Epoch

**UTC** - Coordinated Universal Time




## Типичные задачи при работе со временем

1. Базовые хотелки:
    * Строка/таймстемп <-> Внутренние представление (datetime, ???)
    * Арифметические операции (время между датами, дата через время)
2. Продвинутые хотелки:
    * Работа с таймзонами
    * Нормальная работа с тайзонами, наносекунды, автоматический парсинг и другие проблемы

### datetime

In [1]:
from datetime import datetime

In [2]:
datetime(year=2021, month=10, day=1, hour=18, minute=0, second=1)

datetime.datetime(2021, 10, 1, 18, 0, 1)

In [3]:
d = datetime(2021, 10, 1, 18, 0, 0)
d.year, d.month, d.day, d.hour, d.minute, d.second

(2021, 10, 1, 18, 0, 0)

In [4]:
datetime(2022, 2, 29, 0, 0, 0)

ValueError: day is out of range for month

### datetime → strftime/strptime

In [ ]:
# Регулярно видим в ДЗ

d = "2020-10-01T18:00:01"
year = int(d[:4])
month = int(d[5:7])
day = int(d[8:10])
hour = int(d[11:13])
minute = int(d[14:16])
second = int(d[17:19])
datetime(year, month, day, hour, minute, second)

In [5]:
# А лучше так

datetime.strptime("2021-10-01T18:00:01", "%Y-%m-%dT%H:%M:%S")

datetime.datetime(2021, 10, 1, 18, 0, 1)

In [11]:
dt = datetime(year=2020, month=10, day=6, hour=18, minute=0, second=1)
dt.strftime("%Y-%m-%dT%H:%M:%S")

'2020-10-06T18:00:01'

### datetime → now!

In [9]:
datetime.now()

datetime.datetime(2024, 10, 8, 19, 41, 25, 132438)

### datetime → timedelta

In [12]:
from datetime import datetime, timedelta


dt = datetime.strptime("2022-10-01T18:00:01", "%Y-%m-%dT%H:%M:%S")
td = datetime.now() - dt
td  # а где месяцы?

datetime.timedelta(days=738, seconds=6240, microseconds=668302)

In [13]:
td, td * 2, td / 2

(datetime.timedelta(days=738, seconds=6240, microseconds=668302),
 datetime.timedelta(days=1476, seconds=12481, microseconds=336604),
 datetime.timedelta(days=369, seconds=3120, microseconds=334151))

In [14]:
dt + td, dt - td

(datetime.datetime(2024, 10, 8, 19, 44, 1, 668302),
 datetime.datetime(2020, 9, 23, 16, 16, 0, 331698))

In [15]:
td = timedelta(days=1, hours=10, seconds=10)
td

datetime.timedelta(days=1, seconds=36010)

In [16]:
td.seconds, td.total_seconds()  # не путайте (!)

(36010, 122410.0)

### datetime → timestamp

In [17]:
datetime.now().timestamp()

1728409599.668874

In [18]:
dt = datetime(2022, 10, 10, 18, 0, 0)
dt.timestamp()

1665417600.0

# Вопрос

In [22]:
# Как задать дату, которая будет соответствовать нулевому таймстемпу?
dt = datetime(year=1970, month=1, day=1)
dt.timestamp()

-3600.0

### datetime → Naive/Aware

Naive datetime

In [23]:
dt = datetime(1970, 1, 1, 0, 0, 0)
dt.tzinfo is None

True

In [24]:
dt.timestamp()

-3600.0

Aware datetime

In [25]:
import zoneinfo  # new in 3.9

TZ = zoneinfo.ZoneInfo("UTC")

dt = datetime(1970, 1, 1, 0, 0, 0, tzinfo=TZ)
dt.tzinfo

zoneinfo.ZoneInfo(key='UTC')

In [26]:
dt.timestamp()

0.0

### datetime → Mixing aware and naive

In [27]:
from datetime import datetime
import zoneinfo

dt_moscow = datetime(1970, 1, 1, 3, 0, 0)
dt_utc = datetime(1970, 1, 1, 0, 0, 0, tzinfo=zoneinfo.ZoneInfo("UTC"))

dt_utc - dt_moscow  # что получим?

TypeError: can't subtract offset-naive and offset-aware datetimes

In [28]:
dt_moscow = datetime(1970, 1, 1, 3, 0, 0, tzinfo=zoneinfo.ZoneInfo("Europe/Moscow"))
dt_utc = datetime(1970, 1, 1, 0, 0, 0, tzinfo=zoneinfo.ZoneInfo("UTC"))

dt_moscow - dt_utc

datetime.timedelta(0)

### datetime → Implicit time extracting problems → now

In [29]:
#### Ломаю локальная таймзону ####

import os
import time

os.environ["TZ"] = "America/Los_Angeles"
time.tzset()

##################################

datetime.now().strftime("%Y-%m-%dT%H:%M:%S")

'2024-10-08T10:51:02'

In [30]:
TZ = zoneinfo.ZoneInfo("Europe/Moscow")

datetime.now(TZ).strftime("%Y-%m-%dT%H:%M:%S")

'2024-10-08T20:52:07'

### datetime → Implicit time extracting problems → fromtimestamp

In [31]:
datetime.fromtimestamp(0)

datetime.datetime(1969, 12, 31, 16, 0)

In [32]:
datetime.fromtimestamp(0).strftime("%Y-%m-%dT%H:%M:%S")

'1969-12-31T16:00:00'

In [33]:
datetime.fromtimestamp(0, tz=TZ)

datetime.datetime(1970, 1, 1, 3, 0, tzinfo=zoneinfo.ZoneInfo(key='Europe/Moscow'))

In [34]:
datetime.fromtimestamp(0, tz=TZ).strftime("%Y-%m-%dT%H:%M:%S")

'1970-01-01T03:00:00'

### datetime → Convert timezone

In [35]:
#### Ломаю локальная таймзону ####

import os
import time

os.environ["TZ"] = "Europe/Moscow"
time.tzset()

##################################

dt = datetime(2022, 10, 1, 0, 0, 0)
TZ = zoneinfo.ZoneInfo("America/Los_Angeles")
dt.replace(tzinfo=TZ), dt.astimezone(TZ)

(datetime.datetime(2022, 10, 1, 0, 0, tzinfo=zoneinfo.ZoneInfo(key='America/Los_Angeles')),
 datetime.datetime(2022, 9, 30, 14, 0, tzinfo=zoneinfo.ZoneInfo(key='America/Los_Angeles')))

## Проблемы стандартного datetime

1. Стандартная работа с таймзонами может приводить к множеству багов
2. Микросекундные ограничения на время и вещественный таймстемп
3. Неудобно парсить строчки
4. При работе с таймзоной нет летнего и зимнего времени
5. ....

## Решение: сторонние библиотеки

1. pandas
2. [pendulum](https://pendulum.eustace.io/)

In [36]:
import pandas as pd

(
    pd.Timestamp("2021-10-01T18:00:01+05:00"),
    pd.Timestamp("2021-10-01 18:00:01.001"),
    pd.Timestamp(1728391229001, unit="ms"),
    pd.Timestamp(1728391229001, unit="ms").value
)

(Timestamp('2021-10-01 18:00:01+0500', tz='UTC+05:00'),
 Timestamp('2021-10-01 18:00:01.001000'),
 Timestamp('2024-10-08 12:40:29.001000'),
 1728391229001000000)

In [37]:
pd.Timedelta("1 day 100 minutes 10 ns"), pd.Timedelta("09:00:00").value

(Timedelta('1 days 01:40:00.000000010'), 32400000000000)

In [38]:
for dt in pd.date_range("2024-10-10", "2024-10-11"):
    print(dt)

2024-10-10 00:00:00
2024-10-11 00:00:00


## Минусы

- некоторые методы будут долгими

In [39]:
%%timeit
datetime.strptime("2021-10-01T18:00:01.001", "%Y-%m-%dT%H:%M:%S.%f")

3.95 µs ± 19.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [40]:
%%timeit
pd.to_datetime("2021-10-01 18:00:01.001") # не используйте to_datetime/to_timedelta для единственного значения

161 µs ± 1.05 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [41]:
%%timeit
pd.Timestamp("2021-10-01 18:00:01.001")

981 ns ± 1.93 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
